<a href="https://colab.research.google.com/github/pchodon/Amegashita/blob/Feature-Integration/main_v17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import multiprocessing
import time
from csv import reader
from google.colab import files
import pandas as pd
import threading
import os
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split


sensory_1_resp = 2  # default 1 Sec
sensory_2_resp = 2  # default 1 Sec
temporal_resp =  1  # default 1 Sec
thalamus_resp =  1  # default 1 Sec
reset = 0
forever = 1
max_sensory_stream = 100
Max_lapse = 8
mTrue = 1
mFalse = 0
w, h = 10, 10;
hipp_first = False
stop_emulation = 0
idle_window = 15

row,column = (4,10)

# row 1 = time
# row 2 = sensor_1
# row_3 = sensor_1
# row_4 = learning

#101213141516171819
#20212223242526272829
#3031323334353637383930

TIME = 1
SENSOR_1 = 2
SENSOR_2 = 3
LEARNING = 4


# send_1_sensor,recv_1_sensor
# send_2_sensor,recv_2_senor

def sensory_1(send_1_sensor,recv_1_sensor,parent_conn,msg):
   
  data = []
  with open('/content/drive/MyDrive/sense_1.csv', 'r') as data:
    df_sensory_1 = pd.read_csv(data)

  i = 0
  while(1):
   msg = df_sensory_1.loc[i]
   send_1_sensor.send(msg[0])
   time.sleep(sensory_1_resp)
   i = i+1
   if(i==max_sensory_stream):
        break
  return  

def sensory_2(send_2_sensor,recv_2_sensor,parent_conn,msg):
  
  data = []
  with open('/content/drive/MyDrive/sense_2.csv', 'r') as data:
    df_sensory_2 = pd.read_csv(data)

  i = 0
  while(1):
   msg = df_sensory_2.loc[i]
   send_2_sensor.send(msg[0])
   time.sleep(sensory_2_resp)
   i = i+1
   if(i==max_sensory_stream):
        break
  return  

def hippocampus(xparent_conn,xchild_conn,msg):
    while(1):
      sense = xparent_conn.recv()
      if((sense == 4)):
          print("Hipp Recvd :  ",sense)
          return
      continue


def Idle(idle_state_send,msg):
  data = []
  with open('/content/drive/MyDrive/sense_3.csv', 'r') as data:
    df_sensory_2 = pd.read_csv(data)

  i = 0
  time.sleep(0)
  msg =9
  msg = df_sensory_2.loc[i]
  idle_state_send.send(msg[0])

  while(1):
       time.sleep(idle_window)
       msg = df_sensory_2.loc[i]
       idle_state_send.send(msg[0])
      
     


def thalamus(idle_state_recv,recv_1_sensor,recv_2_sensor,parent_conn,child_conn,xparent_conn,xchild_conn,msg):
  maxim = 10
  lapse = 0
  window = False
  source = False
  sense = 0
  sensor_1,sensor_2 = [3,10]
  # row 1 time
  # row 2 sensor_1
  
  # row 3 sensor_2
  # Episode is 10 frames


  window_start = 0
  while(True):
      sense = idle_state_recv.recv()
      if((sense == 9)):
         print("________Window starts here____________________")
         break
   
  
  while(True):
      #time.sleep(temporal_resp)
      sense_1 = recv_1_sensor.recv()
      print('sense 1 : ',sense_1)
      sense_2 = recv_2_sensor.recv()
      print('sense 2 : ',sense_2)

      sense = sense_1 or sense_2
      
      if((sense == '1') or (sense =='0')):
          if(window_start ==1):
            print("__________Stop thinking__________________")
            window_start = 0
            pass
      else:
         if(window_start == 0):
           window_start = 1    
           print("__________Start thining__________________")
#
      TIME = 1




#
           pass
         else:
           pass 
  
         
      continue


# As sensory input is received
# Start a timer window  
# Reset the window e experiences no sensory inputs


if __name__ == "__main__":
  print("System Starting......")
  msg = 0

        # creating common pipe name
  parent_conn, child_conn = multiprocessing.Pipe()
  send_1_sensor,recv_1_sensor = multiprocessing.Pipe()
  send_2_sensor,recv_2_sensor = multiprocessing.Pipe()
  idle_state_send,idle_state_recv = multiprocessing.Pipe()
  xparent_conn, xchild_conn = multiprocessing.Pipe()
  timer = multiprocessing.Pipe()

  p1 = multiprocessing.Process(target=sensory_1, args=(send_1_sensor,recv_1_sensor,parent_conn,msg))
  p2 = multiprocessing.Process(target=sensory_2, args=(send_2_sensor,recv_2_sensor,parent_conn,msg))
  p3 = multiprocessing.Process(target=thalamus, args=(idle_state_recv,recv_1_sensor,recv_2_sensor,parent_conn,child_conn,xparent_conn, xchild_conn,msg))
  p4 = multiprocessing.Process(target=hippocampus, args=(xparent_conn,xchild_conn,msg))
  p5 = multiprocessing.Process(target=Idle, args=(idle_state_send,msg))



       # running processes
  p1.start()
  p2.start()
  p3.start()
  p4.start()
  p5.start()
 
  time.sleep(100)
  
  print("Starting new cycle")
  p5.join()
  p4.join()
  p3.join()
  p1.join()
  p2.join()

  time.sleep(2)

print("Starting new episode")
#333444

System Starting......
________Window starts here____________________
sense 1 :  1
sense 2 :  0
sense 1 :  1
sense 2 :  0
sense 1 :  1
sense 2 :  0


Process Process-104:
Process Process-105:
Process Process-103:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Process Process-101:
Process Process-102:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "<ipython-input-22-1c784159ab1c>", line 67, in hippocampus
    sense = xparent_conn.recv()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args,

KeyboardInterrupt: ignored

  File "<ipython-input-22-1c784159ab1c>", line 59, in sensory_2
    time.sleep(sensory_2_resp)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
KeyboardInterrupt
KeyboardInterrupt
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
